# Solving Problems Numerically

In this session we will use implement numerical algorithms to solve problems,
that cannot be determined analytically, by resolving our set of equations for a
specific variable. We will learn

- how to implement a different type of numerical root finding algorithm, either
  by writing the respective method ourselves or by using methods from the
  python library python and 
- how to apply this to the same problems from the previous session,

In this context, we will see that a change of boundary conditions may require
to rewrite the formulation of the problem. Therefore, we will go one step
beyond implementing basic root finding methods and learn 

- how we can generalize our problem formulation in a way, that rewriting the
  code of our problem is minimized.

With these learnings we will also better understand, what happens in the back
end of respecitve software employing these types of sultion strategies.

## Root Finding Algorithms

In the previous section we have already learned how to implement a very basic
root finding algorhtim, namely the bisectional search. While it is an easy to
implement and understand approach it does not perform well in terms of 
computational speed. Next to bisecional search (which is a bracketing method)
there is a large variety of other algorithms available, these are for example

- bracketing methods, e.g.

  - Regula Falsi,
  - Interpolate Truncate and Project method,

- iterative methods, e.g.

  - Newton's method, 
  - Secant method or

- combinations of these, e.g.

  - Brent's method

In this session we will have a closer look at Newton's method specifically, but
we will also learn, how we can make use of the python library scipy to
implement a large variety of root finding algorithms easily.

First, go back to the last part of the previous session and check how many
iterations were required to find the pressure ratio leading to the desired
temperature at outlet of the compressor.

In [298]:
# use CoolProp
from CoolProp.CoolProp import PropsSI as props

# fluid information
fluid = 'air'

# inlet conditions
m_1 = 10 # mass flow in kg/s
p_1 = 1 * 10**5 # inlet pressure in Pa
t_1 = 293.15 #inlet temperature in K

# compressor data
eta_c = 0.9 # compressor isentropic efficiency

# outlet conditions
t_2_target = 573.15 # target compressor outlet tempertature in K


# calculate missing properties
h_1 = props('H', 'P', p_1, 'T', t_1, fluid)
s_1 = props('S', 'P', p_1, 'H', h_1, fluid)

# calculate outlet temperature
def calc_t_out_pr(pr, eta, p_in, h_in, medium):
    '''
    function for calculating outlet temperatures
    as a function of pressure ratio
    '''
    s_in = props('S', 'P', p_in, 'H', h_in, medium)
    h_out_s = props('H', 'P', p_in*pr, 'S', s_in, medium)
    h_out = h_in + (h_out_s - h_in) / eta
    t_out = props('T', 'P', p_in*pr, 'H', h_out, medium)
    return t_out


# guess for the pressure ratio
pr_guess = 4
calc_t_out_pr(pr_guess, eta_c, p_1, h_1, fluid)

450.51341458937446

Now reimplement the same problem using Newton's method. The iteration rule for
Newton's method is the following:

$$
x_{i + 1} = x_{i} - \frac{f\left(x_i\right)}{\frac{df}{dx_i}}
$$

The iteration is repeated until the residual value of the equation $f$ is
below the desired threshold of accuracy $\varepsilon$.

$$|f\left(x\right)|\leq \varepsilon$$

To apply this to our use case, we have to

- provide a starting value $x_0$ for $x$,
- calculate the residual value of the function $f$ and
- determine the derivative of the function $\frac{df}{dx}$ with respect to $x$.

For that, the residual function $f$ needs be defined in a way, that the desired
result is zero. In context of our problem, that means we have to calculate the
difference between the target outlet temperature $T_\text{2,target}$ and the
temperature that is given as a result from the calculation of the
`calc_t_out_pr` method.

$$
0 = f\left(\Pi_\text{C}\right) - T_\text{2,target}
$$

It is not trivial to obtain the derivative of the function towards
$\Pi_\text{C}$. However, it is possible to approximate the derivative
numerically using a finite difference, e.g. the central difference:

$$
\frac{\Delta f}{\Delta x} =
\frac{f\left(x+\Delta\right) - f\left(x-\Delta\right)}{2 \cdot \Delta}
$$

Also check how many iterations are necessary, and more interestingly, how many
function evaluations are necessary to find the result.

In [299]:
d = 1e-2  # Delta for change of variable pr

while True:

    residual = calc_t_out_pr(pr_guess, eta_c, p_1, h_1, fluid) - t_2_target
    derivative = (
        calc_t_out_pr(pr_guess + d, eta_c, p_1, h_1, fluid)
         - calc_t_out_pr(pr_guess - d, eta_c, p_1, h_1, fluid)
    ) / (2 * d)

    increment = residual / derivative
    pr_guess -= increment

    # stop the simulation once the difference between the target temperature
    # and the actual temperature is small enough
    if abs(residual) < 1e-6:
        break

pr_guess

9.005367920166826

With the availabilty of many software packages in the python universe, it is
convenient to implement such algorithms with the help of external libraries.
For example, scipy `cite`{scipy} offers a large variety of
[root finding methods](https://docs.scipy.org/doc/scipy/reference/optimize.html#root-finding).

Use the `brentq` method to implement the same procedure.

In [300]:
def residual_function(pr_guess, eta_c, p_1, h_1, fluid, t_2_target):
    return calc_t_out_pr(pr_guess, eta_c, p_1, h_1, fluid) - t_2_target


from scipy.optimize import brentq


brentq(
    residual_function,  # residual function
    a=1, # lower bracket
    b=20, # upper bracket
    # additional arguments after the first one
    args=(eta_c, p_1, h_1, fluid, t_2_target),
)

9.005367920166796

We have seen that we can easily implement the root finding in python to
calculate results of a problem, that cannot be easily analytically solved. 
We can also reuse the implementations we have done in certain situations:

Consider, that the pressure ratio is already fixed to a specific value. We now
want to know what the isentropic efficiency of the compressor must at least be
to ensure that the outlet temperature does not exceed a specified upper limit.
For this, there is two things we can do:

- Either we make a reimplementation of the code, because now all parameters can
  be calculated explicitly without the need of any iteration as the inlet and
  the outlet state are full known.
- Or, we can directly reuse the just implemented method and just modify the
  variable the algorithm should solve for.
  
With the second approach, reimplement the `newton` or `brentq` root finding to
identify at which isentropic efficiency the outlet temperature of 300 °C is not
exceeded, when the pressure ratio is 8.

In [301]:
def residual_function_for_eta(eta_c, pr_guess, p_1, h_1, fluid, t_2_target):
    return calc_t_out_pr(pr_guess, eta_c, p_1, h_1, fluid) - t_2_target

pr = 8

brentq(
    residual_function_for_eta,  # residual function
    a=0.6, # lower bracket
    b=0.95, # upper bracket
    # additional arguments after the first one
    args=(pr, p_1, h_1, fluid, t_2_target),
)

0.8358848988178827

## Newton's method in the multidimensional space

### Introduction

We can see, that all we need to do is to reorder the arguments in the residual
function and we can reutilze the `calc_t_out_pr` method once again. The limit
of this is, when the actual inputs or outputs of our calculation routine change
in a way, that it is not possible anymore to pass my variable I am trying to
solve for into the function. Then a complete reimplementation of the
calculation procedure is required independently of the fact, if the solution
can be obtained analytically or numerically.

In the spirit of trying to minimze the effort we have to take in such 
situations, it is actually possible to generalize the root finding and making
it adaptive to the specific use case. For this, we will implement 
Newton's method in the multidimensional space: The algorithm is supposed to 
find the solution of a set of `n` equations with `n` variables simultaneously.

First, we will define the set of variables we want to solve for and then we
will define the equations that are relevant for our problem. To not introduce
a second new dimenson of complexity in our model, we will keep the compressor
model. As set of variables we will use the state of the fluid at the inlet and
at the outlet of the compressor, specifically

- mass flow $\dot m$,
- pressure $p$ and
- enthalpy $h$.

The reason, why we chose these as set of variables is, that all relevant
component parameters can be derived from these states. Additionally, all
fluid properties, e.g. entropy or temperature, can also be derived from
pressure and enthalpy.

To port Newton's method to the multidimensional space, we have to modify the
iteration procedure:

- the variable $x$ becomes a vector of variables $\vec x$
- the residual value of the function $f\left(x\right)$ becomes a vector of
  residual values $\vec f\left(\vec x \right)$
- the derivative of the function $\frac{df}{dx}$ becomes all partial 
  partial derivatives of the set of functions towards all variables in the
  vector $\vec x$, which is the Jacobian matrix $J\left(\vec x\right)$
- the devision of the residual value by its derivative becomes a matrix
  multiplication of the Jacobian from the left to the residual value vector.

$$
\vec x_{i+1} = \vec x_{i} - J\left(\vec x\right) \cdot f\left(\vec x_{i}\right)
$$

### Application to the compressor model

We have already seen all relevant equations we might want to apply in context
of the compressor model. Note, that we have to write them in the residual form,
meaning, the left side of the equation should always be zero:

- Equality of mass flow at inlet and outlet of compressor: $0 = \dot m_1 - \dot m_2$
- Specified isentropic efficiency of the compressor: $0 = \left(h_\text{2,s} - h_1\right) - \eta_\text{s} \cdot \left(h_2 - h_1\right)$
- Duty of the compressor: $0 = \dot W - \dot m \cdot \left(h_2 - h_1\right)$
- Pressure ratio of the compressor: $0 = \text{pr} \cdot p_1 - p_2$
- Temperature of the fluid at inlet and/or outlet $0 = T - T\left(p,h\right)$
- Volumetric flow of the air at inlet and/or outlet $0 = \dot m \dot v\left(p,h\right)$

The equations for situations, in which one of the variables is directly specified,
e.g. the mass flow or the pressure, are very simple, e.g. for mass flow:
$0 = \text{specification} - \dot m$.

We will create one function for each of the functionalities mentioned above:

In [302]:
def temperature_residual(T, p, h, fluid):
    return props("T", "P", p, "H", h, fluid) - T


def volumetric_flow_residual(v, m, p, h, fluid):
    return m / v - props("D", "P", p, "H", h, fluid)


def duty_residual(W_dot, m, h_in, h_out):
    return W_dot - m * (h_out - h_in)


def pr_residual(pr, p_in, p_out):
    return p_in * pr - p_out


def eta_s_residual(eta_s, p_in, h_in, p_out, h_out, fluid):
    s_in = props("S", "H", h_in, "P", p_in, fluid)
    h_out_s = props("H", "S", s_in, "P", p_out, fluid)
    return eta_s * (h_out - h_in) - (h_out_s - h_in)


def variable_specified_residual(specification, var):
    return specification - var


def variable_equality_residual(var_1, var_2):
    return var_1 - var_2

In [303]:

import numpy as np


dot_m = 5
dot_W = 1.5e6
p_in = 1e5
pr = 10
eta_s = 0.85

x = np.array([
    5, 1e5, 1e5, 5, 10e5, 2e5
])

residuals are function evaluations

In [304]:
residual = np.array([
    variable_specified_residual(dot_m, x[0]),
    variable_specified_residual(p_in, x[1]),
    variable_equality_residual(x[0], x[3]),
    duty_residual(dot_W, x[0], x[2], x[5]),
    pr_residual(pr, x[1], x[4]),
    eta_s_residual(eta_s, x[1], x[2], x[4], x[5], fluid)
])

- simple derivatives we can just place in there

In [305]:
jacobian = np.zeros((6, 6))

jacobian[0, 0] = -1
jacobian[1, 1] = -1
jacobian[2, [0, 3]] = [1, -1]
jacobian[3, [0, 2, 5]] = [x[2] - x[5], x[0], -x[0]]
jacobian[4, [1, 4]] = [pr, -1]

- Complex derivatives -> use finite difference methods

In [306]:
def eta_s_jacobian(eta_s, p_in, h_in, p_out, h_out, fluid):
    d = 1e-1
    return [
        (eta_s_residual(eta_s, p_in + d, h_in, p_out, h_out, fluid)
        - eta_s_residual(eta_s, p_in - d, h_in, p_out, h_out, fluid))
        / (2 * d),
        (eta_s_residual(eta_s, p_in, h_in + d, p_out, h_out, fluid)
        - eta_s_residual(eta_s, p_in, h_in - d, p_out, h_out, fluid))
        / (2 * d),
        (eta_s_residual(eta_s, p_in, h_in, p_out + d, h_out, fluid)
        - eta_s_residual(eta_s, p_in, h_in, p_out - d, h_out, fluid))
        / (2 * d),
        eta_s
    ]


jacobian[5, [1, 2, 4, 5]] = eta_s_jacobian(eta_s, x[1], x[2], x[4], x[5], fluid)

In [307]:
increment = np.linalg.inv(jacobian).dot(residual)
x -= increment
x

array([5.00000000e+00, 1.00000000e+05, 7.72514767e+05, 5.00000000e+00,
       1.00000000e+06, 1.07251477e+06])

The individual parts can now be executed in a loop until convergence is
achieved, which can be measured by the residual vector norm and/or the
increment vector norm.

In [308]:
while True:

    residual = np.array([
        variable_specified_residual(dot_m, x[0]),
        variable_specified_residual(p_in, x[1]),
        variable_equality_residual(x[0], x[3]),
        duty_residual(dot_W, x[0], x[2], x[5]),
        pr_residual(pr, x[1], x[4]),
        eta_s_residual(eta_s, x[1], x[2], x[4], x[5], fluid)
    ])

    jacobian = np.zeros((6, 6))

    jacobian[0, 0] = -1
    jacobian[1, 1] = -1
    jacobian[2, [0, 3]] = [1, -1]
    jacobian[3, [0, 2, 5]] = [x[2] - x[5], x[0], -x[0]]
    jacobian[4, [1, 4]] = [pr, -1]

    jacobian[5, [1, 2, 4, 5]] = eta_s_jacobian(eta_s, x[1], x[2], x[4], x[5], fluid)

    increment = np.linalg.inv(jacobian).dot(residual)
    x -= increment

    if np.linalg.norm(increment) < 1e-6:
        break

x

array([5.00000000e+00, 1.00000000e+05, 3.98908999e+05, 5.00000000e+00,
       1.00000000e+06, 6.98908999e+05])

We can calculate resulting properties from the state vector, e.g. the inlet or
the outlet temperature values.

In [309]:
props("T", "P", x[1], "H", x[2], fluid), props("T", "P", x[4], "H", x[5], fluid)

(272.77035658561897, 567.2531943968664)

To change input values we now only have to exchange the respective equations
when calculating the residual vector and the jacobian matrix. For example, we
can specify the temperature at inlet and at outlet as in the previous example.
For this, we also have to add the partial derivatives of the temperature 
equation. They can be calculated numerically in a similar style as we have done
for the isentropic efficiency equation.

In [310]:
def temperature_jacobian(p, h, fluid):
    return [
        (props("T", "P", p + d, "H", h, fluid) - props("T", "P", p - d, "H", h, fluid)) / (2 * d),
        (props("T", "P", p, "H", h + d, fluid) - props("T", "P", p, "H", h - d, fluid)) / (2 * d)
    ]


In [311]:

while True:

    residual = np.array([
        variable_specified_residual(dot_m, x[0]),
        variable_specified_residual(p_in, x[1]),
        variable_equality_residual(x[0], x[3]),
        # exchange of two equations
        temperature_residual(t_1, x[1], x[2], fluid),
        temperature_residual(t_2_target, x[4], x[5], fluid),

        eta_s_residual(eta_s, x[1], x[2], x[4], x[5], fluid),
    ])

    jacobian = np.zeros((6, 6))

    jacobian[0, 0] = -1
    jacobian[1, 1] = -1
    jacobian[2, [0, 3]] = [1, -1]

    # exchange of two equations
    jacobian[3, [1, 2]] = temperature_jacobian(x[1], x[2], fluid)
    jacobian[4, [4, 5]] = temperature_jacobian(x[4], x[5], fluid)

    jacobian[5, [1, 2, 4, 5]] = eta_s_jacobian(eta_s, x[1], x[2], x[4], x[5], fluid)

    increment = np.linalg.inv(jacobian).dot(residual)
    x -= increment

    if np.linalg.norm(increment) < 1e-6:
        break

x

array([5.00000000e+00, 1.00000000e+05, 4.19408070e+05, 5.00000000e+00,
       8.21404212e+05, 7.05121987e+05])

Now check the results, e.g. pressure ratio or duty.

In [312]:
x[4] / x[1]

np.float64(8.214042118486915)

In [313]:
x[0] * (x[5] - x[2])

np.float64(1428569.5803835187)

In another change of specification, we could exchange mass flow at inlet with
volumetric flow. Here we have to define the partial derivatives as well. For
mass flow it is very simple, for pressure and enthalpy they are the negative
values of the partial derivatives of density to those two, since the mass flow
term is a constant.

In [314]:
def volumetric_flow_jacobian(v, m, p, h, fluid):
    return [
        1 / v,
        - (props("D", "P", p + d, "H", h, fluid) - props("D", "P", p - d, "H", h, fluid)) / (2 * d),
        - (props("D", "P", p, "H", h + d, fluid) - props("D", "P", p, "H", h - d, fluid)) / (2 * d)
    ]

In [315]:
volumetric_flow_residual(4.205, x[0], x[1], x[2], fluid)
x

array([5.00000000e+00, 1.00000000e+05, 4.19408070e+05, 5.00000000e+00,
       8.21404212e+05, 7.05121987e+05])

In [316]:
while True:

    residual = np.array([
        volumetric_flow_residual(10, x[0], x[1], x[2], fluid),
        variable_specified_residual(p_in, x[1]),
        variable_equality_residual(x[0], x[3]),
        # exchange of two equations
        temperature_residual(t_1, x[1], x[2], fluid),
        temperature_residual(t_2_target, x[4], x[5], fluid),

        eta_s_residual(eta_s, x[1], x[2], x[4], x[5], fluid),
    ])

    jacobian = np.zeros((6, 6))
    jacobian[0, [0, 1, 2]] = volumetric_flow_jacobian(10, x[0], x[1], x[2], fluid)
    # print(jacobian[0])
    jacobian[1, 1] = -1
    jacobian[2, [0, 3]] = [1, -1]

    # exchange of two equations
    jacobian[3, [1, 2]] = temperature_jacobian(x[1], x[2], fluid)
    jacobian[4, [4, 5]] = temperature_jacobian(x[4], x[5], fluid)

    jacobian[5, [1, 2, 4, 5]] = eta_s_jacobian(eta_s, x[1], x[2], x[4], x[5], fluid)

    increment = np.linalg.inv(jacobian).dot(residual)
    x -= increment
    norm = np.linalg.norm(residual)

    if norm < 1e-3:
        break

x

array([1.18881747e+01, 1.00000000e+05, 4.19408070e+05, 1.18881747e+01,
       8.21404212e+05, 7.05121987e+05])